In [1]:
import json
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from interpolation import interpolate_mmx, create_patterns
from converters import convert_rp5_to_mmx, convert_raw_to_mmx, convert_raw_to_mmx
from weather_conditions import is_heavy_loose_snow, is_light_loose_snow, is_frost, \
                               is_ice_crusted_ground, is_packed_snow, is_sleet
from loaders import load_mm94_stations
from constants import data_directory
from shutil import copyfile
from tqdm import tqdm_notebook

## Loading station data

In [73]:
mm94_station_id = 1836

raw = load_mm94_stations(mm94_station_id)
mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis)
roadcast = copy(mmx_rwis_interpolated)

## Calculation of dangerous weather conditions

In [74]:
%%time
roadcast['state_heavy_loose_snow'] = is_heavy_loose_snow(roadcast)
roadcast['state_light_loose_snow'] = is_light_loose_snow(roadcast)
roadcast['state_ice_crusted_ground'] = is_ice_crusted_ground(roadcast)
roadcast['state_packed_snow'] = is_packed_snow(roadcast)
roadcast['state_sleet'] = is_sleet(roadcast)
roadcast['state_frost'] = is_frost(roadcast)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexes/base.py:3033: RuntimeWarning: Cannot compare type 'Timestamp' with type 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


CPU times: user 52.1 s, sys: 0 ns, total: 52.1 s
Wall time: 52.2 s


## Loading pathes to pictures

In [75]:
mm94_stations_def = pd.read_csv(data_directory + '/stations_mm94_def.csv')
video_id = mm94_stations_def.loc[mm94_stations_def['station_id'] == mm94_station_id, 'video_id'].values[0]
video_timezone = mm94_stations_def.loc[mm94_stations_def['station_id'] == mm94_station_id, 'video_timezone'].values[0]
video_id

6936

In [76]:
df_video = pd.read_csv(data_directory + '/VIDEO/'+ str(video_id) + '_video.csv', parse_dates=True)
df_video['date_time_utc'] = pd.to_datetime(df_video['date_time_utc'])
df_video.head()

,full_path,date_time,video_dir,path,video_id,date_time_utc
0,/mnt/HARD/MinMax94/data/Video/m18/1149km/2012/...,2012-10-19 11:22:21,m18/1149km,2012/10/19/image12-10-19_11-22-21-69.jpg,6936.0,2012-10-19 08:22:21
1,/mnt/HARD/MinMax94/data/Video/m18/1149km/2012/...,2012-10-19 11:27:20,m18/1149km,2012/10/19/image12-10-19_11-27-20-94.jpg,6936.0,2012-10-19 08:27:20
2,/mnt/HARD/MinMax94/data/Video/m18/1149km/2012/...,2012-10-19 11:32:01,m18/1149km,2012/10/19/image12-10-19_11-32-01-90.jpg,6936.0,2012-10-19 08:32:01
3,/mnt/HARD/MinMax94/data/Video/m18/1149km/2012/...,2012-10-19 12:02:01,m18/1149km,2012/10/19/image12-10-19_12-02-01-92.jpg,6936.0,2012-10-19 09:02:01
4,/mnt/HARD/MinMax94/data/Video/m18/1149km/2012/...,2012-10-19 12:09:12,m18/1149km,2012/10/19/image12-10-19_12-09-12-97.jpg,6936.0,2012-10-19 09:09:12


## Dangerous state photos

In [77]:
events = ['packed_snow', 'sleet', 'frost', 'ice_crusted_ground', 'heavy_loose_snow', 'light_loose_snow']
for state in events:
    state_dt = roadcast.loc[roadcast['state_{0}'.format(state)], 'date_time_utc']
    state_list = []
    gap = pd.Timedelta('1h')
    
    try:
        for dt in state_dt.iteritems():
            mask = (df_video['date_time_utc'] <= (dt[1] + gap)) & (df_video['date_time_utc'] >= (dt[1] - gap))
            state_pathes = list(df_video.loc[mask, 'full_path'].values)
            state_list.extend(state_pathes)

        fraction = len(state_list) // 100
        print(state, len(state_list), fraction)

        state_list_read = state_list[::fraction+1]
        state_list_write = ['/mnt/HARD/MinMax94/data/Photos/{0}/{1}'.format(state, elem.split('/')[-1]) for elem in state_list_read]
        state_list = list(zip(state_list_read, state_list_write))

        for path_read, path_write in tqdm_notebook(state_list):
            copyfile(path_read, path_write)
    except:
        print(state, len(state_list), 'error')

packed_snow 37659 376


Widget Javascript not detected.  It may not be installed or enabled properly.



sleet 2266 22


Widget Javascript not detected.  It may not be installed or enabled properly.



frost 9532 95


...

Widget Javascript not detected.  It may not be installed or enabled properly.



heavy_loose_snow 1517 15


Widget Javascript not detected.  It may not be installed or enabled properly.



light_loose_snow 10509 105


Widget Javascript not detected.  It may not be installed or enabled properly.


In [23]:
#state = 'packed_snow'
#state = 'sleet'
#state = 'frost'
#state = 'ice_crusted_ground'
#state = 'heavy_loose_snow'
#state = 'light_loose_snow'
events = ['packed_snow', 'sleet', 'frost', 'ice_crusted_ground', 'heavy_loose_snow', 'light_loose_snow']
state_dt = roadcast.loc[roadcast['state_{0}'.format(state)], 'date_time_utc']
state_list = []
gap = pd.Timedelta('1h')

for dt in state_dt.iteritems():
    mask = (df_video['date_time_utc'] <= (dt[1] + gap)) & (df_video['date_time_utc'] >= (dt[1] - gap))
    state_pathes = list(df_video.loc[mask, 'full_path'].values)
    state_list.extend(state_pathes)
fraction = len(state_list) // 100
print(len(state_list), fraction)

16350 163


In [24]:
state_list_read = state_list[::fraction]
state_list_write = ['/mnt/HARD/MinMax94/data/Photos/{0}/{1}'.format(state, elem.split('/')[-1]) for elem in state_list_read]
state_list = list(zip(state_list_read, state_list_write))

In [25]:
from shutil import copyfile
from tqdm import tqdm_notebook

for path_read, path_write in tqdm_notebook(state_list):
    copyfile(path_read, path_write)